# Model Training and Validation Pipeline
Now that you have created features, you can use them to train one or more models. In this section, you will generate feature vectors with multiple features from one or more feature sets and feed them into an automated ML training and testing pipeline to create high-quality models.

The ML pipeline can be triggered and tracked manually during the interactive devel‐ opment, or it can be saved (into Git) and be executed automatically on a given schedule or as a reaction to different events (such as code modification, CI/CD, data changes, model drift, and so on). See [MLRun project and CI/CD documentation](https://docs.mlrun.org/en/stable/projects/project.html) for details.


### Saving and loading projects from GIT

After you saved your project and its elements (functions, workflows, artifacts, etc.) you can commit all your changes to a 
GIT repository. This can be done using standard GIT tools or using MLRun `project` methods such as `pull`, `push`, 
`remote`, which calls the Git API for you.

Projects can then be loaded from Git using MLRun `load_project` method, for example: 

    project = mlrun.load_project("./myproj", "git://github.com/mlrun/project-demo.git", name=project_name)
    
or using MLRun CLI:

    mlrun project -n myproj -u "git://github.com/mlrun/project-demo.git" ./myproj
    
Projects can be loaded or created by using MLRun `get_or_create_project` method.
    
Read [CI/CD integration](../../projects/ci-integration.html) for more details.

In [10]:
import mlrun
project = mlrun.get_or_create_project(
    name="fraud-demo",
    context="./",
    user_project=True,
    )

> 2024-12-06 08:52:19,869 [info] Project loaded successfully: {"project_name":"fraud-demo-jovyan"}


## Creating and Evaluating a Feature Vector

Models are trained with multiple features, which can arrive from different feature sets and be collected into training (feature) vectors. Feature stores know how to correctly combine the features into a vector by implementing smart JOINs and assessing the time dimension (time traveling).
To define a feature vector, you need to specify a name, the list of features it contains, the target features (labels), and other optional parameters. Features are specified as `<FeatureSet>.<Feature> or <FeatureSet>.*`  (all the features in a feature set). The following part demonstrates how to create and use a feature vector.


### Create a feature vector

In [11]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the list of features to use
features = ['events.*',
            'transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_sum_14d', 
            'transactions.es_health_sum_14d',
            'transactions.es_otherservices_sum_14d', 
            'transactions.es_food_sum_14d',
            'transactions.es_hotelservices_sum_14d', 
            'transactions.es_barsandrestaurants_sum_14d',
            'transactions.es_tech_sum_14d', 
            'transactions.es_sportsandtoys_sum_14d',
            'transactions.es_wellnessandbeauty_sum_14d', 
            'transactions.es_hyper_sum_14d',
            'transactions.es_fashion_sum_14d', 
            'transactions.es_home_sum_14d', 
            'transactions.es_travel_sum_14d', 
            'transactions.es_leisure_sum_14d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week']

### Create a feature vector

In [12]:
# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using the feature store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the feature store
transactions_fv.save()

In [20]:
transactions_fv.to_dataframe()

MLRunNotFoundError: there are no offline targets for this feature vector

In [15]:
print(transactions_fv.save())

None


In [13]:
# test get_offline_features function

from mlrun.datastore.targets import ParquetTarget
data = transactions_fv.get_offline_features(target=ParquetTarget())


FileNotFoundError: mlrun/projects/fraud-demo-jovyan/FeatureStore/events/parquet/sets/events/1733393644625_760

In [ ]:
data

In [ ]:
data.to_dataframe().head()

Once you have defined the feature vector, you can use `get_offline_features()` to generate the vector dataset and return it as a dataframe or materialize it into a file (CSV or Parquet). The next part demonstrates how to retrieve a vector, materialize it, and view its results.

## Building and Running an Automated Training and Validation Pipeline

MLRun allows the building of distributed ML pipelines that can handle data processing, automated feature selection, training, optimization, testing, deployments, and so on. Pipelines are composed of steps that run or deploy custom or library (from the MLRun hub) serverless functions. Pipelines can be run locally (for debugging or small-scale tasks), on a scalable Kubernetes cluster (using Kubeflow), or in a CI/CD system.

The example consists of the following pipeline steps (all using pre-defined MLRun hub functions):

1. Materialize a feature vector (using `src/get_vector`). 
2. Select the most optimal features (using `hub://feature_selection`).
3. Train the model with multiple algorithms (using `hub://auto_trainer`).
4. Evaluate the model (using `hub://auto_trainer`).
5. Deploy the model and its application to the test cluster (using `hub://v2_model_server`). The next section will explain the model and application pipeline in detail.

Each step can accept the previous steps’ results or data, and generate results, multiple visual artifacts/charts, versioned data objects, and registered models.

We have defined the workflow in [`src/new_train_workflow.py`](./src/new_train_workflow.py). 

## Running the ML pipeline

The workflow/pipeline can be executed using the MLRun SDK (`project.run()` method) or using CLI commands (mlrun project), and can run directly from the source repo (GIT). See details in [MLRun Projects and Automation documentation](https://docs.mlrun.org/en/stable/projects/project.html).

You can set arguments and destinations for the different artifacts when you run the workflow. The pipeline progress and results are shown in the notebook. Alternatively, you can check the progress, logs, artifacts, and more, in the MLRun UI or the CI/CD system. The next part demonstrates how to run the pipeline with custom arguments using the SDK.

In [18]:
run_id = project.run(
    'main',
    arguments={'vector_name':"transactions-fraud",
               'features': features,
                'label_column':"labels.label",
              }, 
    dirty=True, watch=True)

> 2024-12-06 09:19:48,392 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2024-12-06 09:19:48,415 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-12-06 09:19:48,419 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-12-06 09:19:48,420 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-12-06 09:19:48,421 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2024-12-06 09:19:48,423 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe


> 2024-12-06 09:19:48,426 [info] Storing function: {"db":null,"name":"get-vector","uid":"ec17642e639041f18d1276162f7bb692"}
> 2024-12-06 09:19:48,722 [error] Execution error, Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/mlrun/runtimes/local.py", line 506, in exec_from_params
    val = mlrun.handler(
  File "/opt/conda/lib/python3.9/site-packages/mlrun/package/__init__.py", line 140, in wrapper
    func_outputs = func(*args, **kwargs)
  File "src/get_vector.py", line 28, in get_offline_features
    data = fv.get_offline_features(target=ParquetTarget())
  File "/opt/conda/lib/python3.9/site-packages/mlrun/feature_store/feature_vector.py", line 810, in get_offline_features
    return mlrun.feature_store.api._get_offline_features(
  File "/opt/conda/lib/python3.9/site-packages/mlrun/feature_store/api.py", line 274, in _get_offline_features
    return merger.start(
  File "/opt/conda/lib/python3.9/site-packages/mlrun/feature_store/retrieval/base.py", lin

mlrun/projects/fraud-demo-jovyan/FeatureStore/events/parquet/sets/events/1733393644625_760


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
fraud-demo-jovyan,...2f7bb692,0,Dec 06 09:19:48,error,run,get-vector,workflow=6ac4fd370f2e427781392765f649d542kind=localowner=jovyanhost=mlrun-jupyter-6557dd94b7-mrsm2,,"feature_vector=transactions-fraudfeatures=['events.*', 'transactions.amount_max_2h', 'transactions.amount_sum_2h', 'transactions.amount_count_2h', 'transactions.amount_avg_2h', 'transactions.amount_max_12h', 'transactions.amount_sum_12h', 'transactions.amount_count_12h', 'transactions.amount_avg_12h', 'transactions.amount_max_24h', 'transactions.amount_sum_24h', 'transactions.amount_count_24h', 'transactions.amount_avg_24h', 'transactions.es_transportation_sum_14d', 'transactions.es_health_sum_14d', 'transactions.es_otherservices_sum_14d', 'transactions.es_food_sum_14d', 'transactions.es_hotelservices_sum_14d', 'transactions.es_barsandrestaurants_sum_14d', 'transactions.es_tech_sum_14d', 'transactions.es_sportsandtoys_sum_14d', 'transactions.es_wellnessandbeauty_sum_14d', 'transactions.es_hyper_sum_14d', 'transactions.es_fashion_sum_14d', 'transactions.es_home_sum_14d', 'transactions.es_travel_sum_14d', 'transactions.es_leisure_sum_14d', 'transactions.gender_F', 'transactions.gender_M', 'transactions.step', 'transactions.amount', 'transactions.timestamp_hour', 'transactions.timestamp_day_of_week']label_feature=labels.labeltarget={'name': 'parquet', 'kind': 'parquet'}update_stats=True",


> 2024-12-06 09:19:48,800 [info] Run execution finished: {"name":"get-vector","status":"error"}
> 2024-12-06 09:19:48,801 [error] Workflow run failed: {"exc_info":["Traceback (most recent call last):\n","  File \"/opt/conda/lib/python3.9/site-packages/mlrun/projects/pipelines.py\", line 716, in run\n    workflow_handler(**workflow_spec.args)\n","  File \"./src/train_workflow.py\", line 41, in pipeline\n    get_vector_run = project.run_function(\n","  File \"/opt/conda/lib/python3.9/site-packages/mlrun/projects/project.py\", line 3473, in run_function\n    return run_function(\n","  File \"/opt/conda/lib/python3.9/site-packages/mlrun/projects/operations.py\", line 213, in run_function\n    run_result = function.run(\n","  File \"/opt/conda/lib/python3.9/site-packages/mlrun/runtimes/base.py\", line 403, in run\n    return launcher.launch(\n","  File \"/opt/conda/lib/python3.9/site-packages/mlrun/launcher/local.py\", line 126, in launch\n    result = self._execute(\n","  File \"/opt/conda

## Test the model endpoint


Now that your model is deployed using the pipeline, you can invoke it as usual:

In [10]:
# Define your serving function
serving_fn = project.get_function('serving')

# Choose an id for your test
sample_id = 'C1000148617'
model_inference_path = '/v2/models/fraud/infer'

# Send our sample ID for predcition
serving_fn.invoke(path=model_inference_path,
                  body={'inputs': [[sample_id]]})

> 2024-12-05 11:22:55,328 [warning] Function is in the unhealthy state


ValueError: no function address first run .deploy()

## Done!

You've completed part 4 - the model training with the feature store.
Proceed to [Part 5](06-real-time-serving-pipeline.ipynb) to learn how to deploy real-time application pipelines.